In [1]:
import os
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

DATA_DIR = './data'

data = []
labels = []
counter=0
outliers=0
min_marks=float(np.inf)
for dir in os.listdir(DATA_DIR):
    if counter==27:break
    counter+=1
    for img_path in os.listdir(os.path.join(DATA_DIR, dir)):
        temp_dat = []

        x_dat = []
        y_dat = []

        img = cv2.imread(os.path.join(DATA_DIR, dir, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            c1=0
            for hand_landmarks in results.multi_hand_landmarks:
                # print('loop 1',c1)
                c2=0
                for i in range(len(hand_landmarks.landmark)):
                    # if c2==42/2:break
                    # print('loop 2',c2)
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    c2+=1

                    x_dat.append(x)
                    y_dat.append(y)
                c3=0
                for i in range(len(hand_landmarks.landmark)):
                    # if c3==42/2:break
                    # print('loop 3',c3)
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    temp_dat.append(x - min(x_dat))
                    temp_dat.append(y - min(y_dat))
                    c3+=1
                c1+=1
            if len(temp_dat)==42:
                data.append(temp_dat)
                labels.append(dir)
            else:
                outliers+=1
# kv = {'data': data, 'labels': labels}
print(outliers)

3


In [ ]:
import numpy as np
import pandas as pd
df=pd.DataFrame(data)
df['label']=labels
# df['label'] = df['label'].astype(int)
df=pd.get_dummies(df,columns=['label'])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,label_r,label_s,label_t,label_u,label_v,label_w,label_x,label_y,label_z,label_zz
0,0.025622,0.162825,0.061647,0.140807,0.090377,0.095223,0.095250,0.038278,0.085841,0.0,...,False,False,False,False,False,False,False,False,False,False
1,0.023657,0.163559,0.061296,0.140712,0.090161,0.096142,0.096384,0.039400,0.086492,0.0,...,False,False,False,False,False,False,False,False,False,False
2,0.021938,0.172324,0.062159,0.150878,0.096033,0.108502,0.104907,0.047288,0.097026,0.0,...,False,False,False,False,False,False,False,False,False,False
3,0.023781,0.179952,0.063638,0.157296,0.098190,0.111286,0.107273,0.048136,0.099261,0.0,...,False,False,False,False,False,False,False,False,False,False
4,0.024869,0.179553,0.066026,0.156796,0.100216,0.109375,0.109174,0.046514,0.100014,0.0,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
pca=PCA(0.99)
pca.fit(df.iloc[:,0:42].values)
print(pca.explained_variance_ratio_)

[0.53574646 0.18572476 0.07624138 0.06625424 0.05003651 0.0252917
 0.0207792  0.01463149 0.00582614 0.00547536 0.00362546 0.00238823]


In [5]:
projections=pca.transform(df.iloc[:,0:42].values)
proj_df=pd.DataFrame(projections)
proj_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.253708,-0.051864,-0.068817,-0.018405,0.025159,-0.031754,0.026992,-0.001006,0.023001,-0.008706,0.009841,-0.001854
1,-0.253172,-0.053914,-0.070430,-0.018530,0.025601,-0.032117,0.026602,-0.001152,0.022159,-0.008150,0.010423,-0.000365
2,-0.249158,-0.065186,-0.075908,-0.004237,0.020455,-0.032550,0.046477,0.004677,0.009122,0.001267,0.019736,0.002195
3,-0.233408,-0.058690,-0.077517,-0.001168,0.021328,-0.032546,0.055165,0.003434,0.011149,0.005049,0.019656,-0.000797
4,-0.237392,-0.059504,-0.074453,-0.002308,0.018957,-0.037919,0.056424,0.004197,0.011561,0.003229,0.021175,-0.002518


In [6]:
df.iloc[:,42:len(df.columns)]

,label_a,label_b,label_c,label_d,label_e,label_f,label_g,label_h,label_i,label_j,...,label_r,label_s,label_t,label_u,label_v,label_w,label_x,label_y,label_z,label_zz
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2691,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2692,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2693,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [7]:
x_train, x_test, y_train, y_test = train_test_split(proj_df, df.iloc[:,42:len(df.columns)], test_size=0.2)
model =Sequential()
model.add(Dense(100,input_shape=(12,),activation='relu'))
model.add(Dense(200,activation='relu'))
model.add(Dense(300,activation='relu'))
model.add(Dense(400,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(27,activation='sigmoid'))

model.compile(loss="binary_crossentropy",optimizer='adam',metrics='accuracy')


In [8]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
68/68 [==============================] - 2s 5ms/step - loss: 0.2450 - accuracy: 0.0399
Epoch 2/10
68/68 [==============================] - 0s 6ms/step - loss: 0.1428 - accuracy: 0.1238
Epoch 3/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0903 - accuracy: 0.4606
Epoch 4/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0430 - accuracy: 0.7718
Epoch 5/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0205 - accuracy: 0.9086
Epoch 6/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0136 - accuracy: 0.9402
Epoch 7/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0081 - accuracy: 0.9629
Epoch 8/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0076 - accuracy: 0.9680
Epoch 9/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0073 - accuracy: 0.9624
Epoch 10/10
68/68 [==============================] - 0s 5ms/step - loss: 0.0050 - accuracy: 0.9791


In [9]:
import pickle
f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

In [10]:
print(x_test.iloc[0].shape)

(12,)


In [11]:
model.predict(np.array(x_test.iloc[0]).reshape(1,12))

1/1 [==============================] - 0s 119ms/step


array([[0.0000000e+00, 1.2351586e-06, 3.5636705e-05, 5.0175725e-10,
        1.6872325e-24, 0.0000000e+00, 3.6200823e-27, 1.3328644e-12,
        3.4547818e-34, 8.8990630e-14, 1.0263767e-31, 4.4544238e-20,
        3.2585320e-14, 2.2633216e-15, 6.8668773e-20, 0.0000000e+00,
        8.2120578e-20, 0.0000000e+00, 3.0153972e-33, 0.0000000e+00,
        4.7584320e-21, 2.5017218e-33, 9.2953503e-09, 7.2470561e-36,
        1.9775009e-06, 0.0000000e+00, 9.9941540e-01]], dtype=float32)

In [12]:
y_predict = model.predict(x_test)
def to_lbl(y):
    mx=max(y)
    tf_list=[]
    for i in y:
        if i==mx:tf_list.append(True)
        else:tf_list.append(False)
    return tf_list
for i in range(len(y_predict)):
    y_predict[i]=to_lbl(y_predict[i])
score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))



17/17 [==============================] - 0s 2ms/step
99.25788497217069% of samples were classified correctly !


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1300      
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 dense_2 (Dense)             (None, 300)               60300     
                                                                 
 dense_3 (Dense)             (None, 400)               120400    
                                                                 
 dense_4 (Dense)             (None, 500)               200500    
                                                                 
 dense_5 (Dense)             (None, 27)                13527     
                                                                 
Total params: 416227 (1.59 MB)
Trainable params: 416227 

In [14]:
cap = cv2.VideoCapture(0)
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

labels_dict = {i: chr(65+i) for i in range(26)}
# print(labels_dict)
labels_dict[26]=' '
while True:

    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        # print(data_aux)
        if len(data_aux)==42:
            in_dat=pca.transform(np.array(data_aux).reshape(1,42))
            # print(len(in_dat))
            prediction = model.predict(np.array(in_dat).reshape(1,12))
            print(np.argmax(prediction,axis=1))
            predicted_character = labels_dict[np.argmax(prediction,axis=1)[0]]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                        cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)


cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 104ms/step
[24]
1/1 [==============================] - 0s 26ms/step
[5]
1/1 [==============================] - 0s 21ms/step
[23]
1/1 [==============================] - 0s 28ms/step
[5]
1/1 [==============================] - 0s 23ms/step
[23]
1/1 [==============================] - 0s 22ms/step
[23]
1/1 [==============================] - 0s 23ms/step
[23]
1/1 [==============================] - 0s 21ms/step
[23]
1/1 [==============================] - 0s 23ms/step
[23]
1/1 [==============================] - 0s 22ms/step
[23]
1/1 [==============================] - 0s 22ms/step
[23]
1/1 [==============================] - 0s 22ms/step
[23]
1/1 [==============================] - 0s 24ms/step
[7]
1/1 [==============================] - 0s 21ms/step
[7]
1/1 [==============================] - 0s 20ms/step
[7]
1/1 [==============================] - 0s 22ms/step
[7]
1/1 [==============================] - 0s 22ms/step
[7]
1/1 [==============================] 

KeyboardInterrupt: 

In [15]:
cap.release()
cv2.destroyAllWindows()

: 